In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


# Introduction

## Predict the Introverts from the Extroverts

Here I've been tasked with whether a person is proficient in the lost art of **touching grass** in a daily basis.<br>
It could also be called introvert detection **(only for attention seekers)**<br>
It's now that i wonder whether my model will come out good or not.<br>
All I can say is **LET ME COOK**.

---

### Dataset Overview

The dataset includes:
- Personal details (e.g. age, gender)
- Social interaction metrics
- Survey-based features
- Potentially some categorical and numerical values needing preprocessing

We'll begin by **exploring the data**, followed by **feature engineering**, **model training**, and **performance evaluation**.

---

### Objective

- Understand what factors correlate with introverted behavior
- Build a robust and generalizable prediction model
- Log experiments and model comparisons
- Visualize key insights along the way

---

### Tools & Libraries

We'll be using:
- **Pandas / NumPy** for data manipulation
- **Seaborn / Plotly / Matplotlib** for visualizations
- **Scikit-learn / XGBoost / LightGBM** for modeling
- **MLflow** (optional) to track and compare model runs

---

~Wonder what I actually am.~

# Import Modules

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import KNNImputer

# Load the Dataset

In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s5e7/train.csv")
test = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

In [4]:
# Info of training data
print(train.info())
train.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB
None


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
13752,13752,0.0,No,8.0,5.0,No,12.0,6.0,Extrovert
5008,5008,3.0,No,3.0,6.0,No,9.0,5.0,Extrovert
665,665,5.0,Yes,3.0,3.0,Yes,5.0,0.0,Introvert
8663,8663,3.0,No,10.0,5.0,No,13.0,3.0,Extrovert
8467,8467,2.0,No,8.0,6.0,No,NaN,9.0,Extrovert


In [5]:
# describe the training data
print(test.info())
test.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6175 entries, 0 to 6174
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         6175 non-null   int64  
 1   Time_spent_Alone           5750 non-null   float64
 2   Stage_fear                 5577 non-null   object 
 3   Social_event_attendance    5778 non-null   float64
 4   Going_outside              5709 non-null   float64
 5   Drained_after_socializing  5743 non-null   object 
 6   Friends_circle_size        5825 non-null   float64
 7   Post_frequency             5767 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 386.1+ KB
None


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
2944,21468,3.0,No,7.0,3.0,No,11.0,6.0
3882,22406,2.0,No,7.0,7.0,No,5.0,4.0
194,18718,11.0,Yes,NaN,1.0,Yes,5.0,1.0
924,19448,0.0,No,5.0,6.0,No,5.0,3.0
212,18736,NaN,No,4.0,7.0,No,5.0,8.0


# Exploratory Data Analysis (EDA)

So first of all I wanna say one thing: 

> **There is no such thing as a pureblood introvert or extrovert in this world.**<br>

All of us tread in some form of between there.

I understand that this pill may be hard to swallow for some, so do feel free to bombard the comments with hate. **(I'll at least have some degree of social interaction through this.)**

In [6]:
# Describe the data

# trainset
print("TrainSet:\n",train.describe(include='all'), end = "\n\n")

# trainset
print("TestSet:\n",test.describe(include='all'))

TrainSet:
                   id  Time_spent_Alone Stage_fear  Social_event_attendance  \
count   18524.000000      17334.000000      16631             17344.000000   
unique           NaN               NaN          2                      NaN   
top              NaN               NaN         No                      NaN   
freq             NaN               NaN      12609                      NaN   
mean     9261.500000          3.137764        NaN                 5.265106   
std      5347.562529          3.003786        NaN                 2.753359   
min         0.000000          0.000000        NaN                 0.000000   
25%      4630.750000          1.000000        NaN                 3.000000   
50%      9261.500000          2.000000        NaN                 5.000000   
75%     13892.250000          4.000000        NaN                 8.000000   
max     18523.000000         11.000000        NaN                10.000000   

        Going_outside Drained_after_socializing  Fri

In [7]:
# Lets first check for missing values

#trainset
print("TrainSet: \n\n",train.isnull().sum(), end = "\n\n")

#testset
print("TestSet: \n\n",test.isnull().sum())

TrainSet: 

 id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

TestSet: 

 id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64


Looks like there are quite the number of missing values present in our data, so first we would need to resolve those.

We need to convert all objects data to numbers/boolean.

In [8]:
# encode the object data
train['Stage_fear'] = train['Stage_fear'].map({'No':0 , 'Yes':1})
train['Drained_after_socializing'] = train['Drained_after_socializing'].map({'No':0 , 'Yes':1})
train['Personality'] = train['Personality'].map({'Extrovert':0 , 'Introvert':1})

test['Stage_fear'] = test['Stage_fear'].map({'No':0 , 'Yes':1})
test['Drained_after_socializing'] = test['Drained_after_socializing'].map({'No':0 , 'Yes':1})

# Missing Values Imputation

So how i'm planning to do this is by using corr of the missing features with other features.

Then, i'll use a KNN Imputer to impute the values. We'll see where it goes from that.

In [9]:
# initialize the imputer
imputer = KNNImputer(n_neighbors = 7)

# train the imputer model
imputed = imputer.fit_transform(train)

train = pd.DataFrame(imputed, columns = train.columns)

In [10]:
# Same for test data
imputed = imputer.fit_transform(test)
test = pd.DataFrame(imputed, columns = test.columns)

Now that we have imputed all the missing values in our data, we can start data analysis through plots.

# Modelling

In [11]:
# imports
from xgboost import XGBClassifier 
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split

In [12]:
#Seperate the data]
X = train.drop(['Personality'], axis = 1)
Y = train['Personality']

In [13]:
X_train,X_test, Y_train,Y_test = train_test_split(X,
                                                 Y,
                                                 test_size = 0.2,
                                                 random_state = 42)

In [14]:
#optuna hobjective function
def objective(trial):
    param = {
    'booster':trial.suggest_categorical('booster',['gbtree','dart','gblinear']),
    'objective':'binary:logistic',
    'eval_metric':'auc',
    'tree_method':trial.suggest_categorical('tree_method',['hist','exact','approx']),
    'scale_pos_weight':trial.suggest_float('scale_pos_weight',1,100),
    'max_depth':trial.suggest_int('max_depth',3,12),
    'learning_rate':trial.suggest_float('learning_rate',0.001, 0.3),
    'subsample':trial.suggest_float('subsample',0.5,1.0),
    'colsample_bytree':trial.suggest_float('colsample_bytree', 0.5, 1.0),
    'lambda':trial.suggest_float('lambda',1e-3, 10.0),
    'alpha':trial.suggest_float('alpha',1e-3,10.0)
    }

    model = XGBClassifier(**param, use_label_encoder = False)
    model.fit(X_train, Y_train)
    preds = model.predict(X_test)
    return precision_score(Y_test, preds)

In [ ]:
#create study object
study = optuna.create_study(direction='maximize')
optuna.logging.set_verbosity(optuna.logging.INFO)
study.optimize(objective, n_trials = 100)

[I 2025-08-06 16:56:47,969] A new study created in memory with name: no-name-04a52c8d-2e20-49e8-a09e-d64437558162
[I 2025-08-06 16:56:48,135] Trial 0 finished with value: 0.25695006747638327 and parameters: {'booster': 'gblinear', 'tree_method': 'exact', 'scale_pos_weight': 16.43991670657078, 'max_depth': 9, 'learning_rate': 0.12805920227482603, 'subsample': 0.7489887378537563, 'colsample_bytree': 0.9923593792420256, 'lambda': 5.534366048627258, 'alpha': 3.4890352549500667}. Best is trial 0 with value: 0.25695006747638327.
[I 2025-08-06 16:56:48,613] Trial 1 finished with value: 0.9456289978678039 and parameters: {'booster': 'gbtree', 'tree_method': 'approx', 'scale_pos_weight': 2.2934705987754, 'max_depth': 6, 'learning_rate': 0.25956815347211254, 'subsample': 0.8875076286272443, 'colsample_bytree': 0.8634131010834687, 'lambda': 0.18732642065313623, 'alpha': 6.431997313971458}. Best is trial 1 with value: 0.9456289978678039.
[I 2025-08-06 16:56:51,146] Trial 2 finished with value: 0.8

In [ ]:
# Now we use the best model available from the study object
best_params = study.best_params
model = XGBClassifier(**best_params)
model.fit(X_train,Y_train)
preds = model.predict(X_test)
preds_proba = model.predict_proba(X_test)[:,1]

In [ ]:
print(f"Model: XGBClassifier with hyperparams tuned with Optuna")
print(f"Acuracy Score : {accuracy_score(Y_test, preds)}")
print(f"ROC_AUC Score : {roc_auc_score(Y_test, preds_proba)}")
print(f"Precision Score : {precision_score(Y_test, preds)}")
print(f"Recall Score : {recall_score(Y_test, preds)}")
print(f"Classification Report : \n{classification_report(Y_test, preds)}")
print(f"Confusion Matrix : \n")
cm = confusion_matrix(Y_test, preds)
cm_disp = ConfusionMatrixDisplay(confusion_matrix = cm)
cm_disp.plot()
plt.show()

In [ ]:
# full train
model.fit(X,Y)
preds = model.predict(test)

In [ ]:
output = pd.DataFrame({'id': test.id.astype(int),
                      'Personality': preds})

output['Personality'] = output['Personality'].map({0:'Extrovert', 1:'Introvert'})

output.to_csv('submission.csv', index = False)

In [ ]:
output

In [ ]:
import joblib
joblib.dump(model, 'introvert_classifier_model.pkl')